# Algorithm Design
## Chapter 4 - Greedy Algorithms

Required python packages:

- numpy
- matplotlib

In [56]:
import time
import matplotlib.pyplot as plt


def format_number_string(n):
  """ add commas every 3 digits in a number """
  n = str(n)
  if len(n) <= 3:
    return n
  else:
    return format_number_string(n[:-3]) + "," + n[-3:]

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def args2str(args):
  if len(args) == 0:
    return ''
  elif len(args) == 1:
    return str(args[0])
  return ', '.join([str(x) for x in args])

class UnitTest:
  def __init__(self, fun, test_fun=(lambda x, y: x==y), inplace=False, post_process=None, show_correct=True, early_stop=False, title=""):
    self.fun = fun
    self.post_process = post_process
    self.show_correct = show_correct
    self.test_fun = test_fun
    self.inplace = inplace
    self.title = title
    self.early_stop = early_stop
    self.inputs = []

  def add_input(self, expected_result, *args):
    self.inputs.append((expected_result, args))
    
  def _run_one(self, expected_result, *args):
    import copy
    try:
      if self.inplace:
        saved = copy.deepcopy(args[0])
        self.fun(*args)
        res = args[0]
        args = tuple(saved if i==0 else args[i] for i in range(len(args)))
      else:
        res = self.fun(*args)
      if self.post_process is not None:
        resp = self.post_process(res)
        expected_resultp = self.post_process(expected_result)
      else:
        resp = res
        expected_resultp = expected_result
      if self.test_fun(resp, expected_resultp):
        if self.show_correct:
          print(bcolors.OKGREEN + "âœ”  " + self.fun.__name__ +  "(" + args2str(args) + ") -> " + str(expected_result) + "" + bcolors.ENDC)
        return True
      else:
        print(bcolors.FAIL + "âœ—  " + self.fun.__name__ +  "(" + args2str(args) + ") -> " + str(res) + " (expected: " + str(expected_result) + ")" + bcolors.ENDC)
        return False
    except Exception as e:
      print(bcolors.FAIL + "âœ— " + self.fun.__name__ +  "(" + args2str(args) + ") is incorrect: the execution produced an error" + bcolors.ENDC)
      raise e
    
  def run(self):
    print(bcolors.HEADER + "--- Running test: " + self.title + bcolors.ENDC)
    passed = 0
    for expected_result, args in self.inputs:
      if self._run_one(expected_result, *args):
        passed += 1
      else:
        if self.early_stop:
          break
    if passed == len(self.inputs):
      print(bcolors.OKGREEN + "âœ” " + str(passed) + "/" + str(len(self.inputs)) + " tests passed." + bcolors.ENDC)
    else:
      print(bcolors.FAIL + "âœ— " + str(passed) + "/" + str(len(self.inputs)) + " tests passed." + bcolors.ENDC)
    print(bcolors.HEADER + "--- Test finished." + bcolors.ENDC)
    return passed == len(self.inputs)

class UnitTestImmediate:
  def __init__(self, title=""):
    self.title = title
    self.correct = 0
    self.total = 0

  def start(self):
    print(bcolors.HEADER + "--- Running test: " + self.title + bcolors.ENDC)
    self.correct = 0
    self.total = 0

  def assert_true(self, fun, message, fun_name):
    self.total += 1
    try:
      if fun():
        self.correct += 1
      else:
        print(bcolors.FAIL + "âœ—  Result of '" + fun_name +  "()' is incorrect.\n" + message  + bcolors.ENDC)
    except Exception as e:
      print(bcolors.FAIL + "âœ— Result of '" + fun_name +  "()' is incorrect: the execution produced an error" + bcolors.ENDC)
      raise e
    
  def end(self):
    if self.correct == self.total:
      print(bcolors.OKGREEN + "âœ” " + str(self.correct) + "/" + str(self.total) + " tests passed." + bcolors.ENDC)
    else:
      print(bcolors.FAIL + "âœ— " + str(self.correct) + "/" + str(self.total) + " tests passed." + bcolors.ENDC)
    print(bcolors.HEADER + "--- Test finished." + bcolors.ENDC)
    return self.correct == self.total



def test_runtime(sort_fun, generator, sizes, iter=3):
  """
  Function to test the runtime of a function
  """
  res = []
  for n in sizes:
    total = 0
    for i in range(iter):
      array = generator(n)
      t1 = time.time()
      sort_fun(array)
      t2 = time.time()
      total += t2-t1
    total = total / iter * 1000
    res.append(total)
    print("\t\tarray size:", format_number_string(n), "-> time", format(total, '.3f') , "s" )
  return res

def unit_test_sort(sort_function, inplace):
  import numpy as np
  import dis
  np.set_printoptions(precision=2)
  np.random.seed(0)
  unitTest = UnitTest(sort_function, test_fun=lambda x, y: np.all(x == y), inplace=inplace)

  for i in range(5):
    array = np.random.rand(10)
    sorted = np.sort(array)
    unitTest.add_input(sorted, array.copy())
    if i == 0:
      np.random.seed()

  return unitTest.run()

def runtime_test_sort(sort_function, input_sizes):
  import numpy as np
  import sys
  sys.setrecursionlimit(10**6)
  #sizes = [2**i for i in range(6, 12)]

  print(bcolors.HEADER + "--- Running runtime test for '" + sort_function.__name__ +  "':" + bcolors.ENDC)

  print(bcolors.OKBLUE + "\t - Input is a sorted array:" + bcolors.ENDC)
  res_sorted = test_runtime(sort_function, np.arange, input_sizes)
  print(bcolors.OKBLUE + "\t - Input is a reverse sorted array:" + bcolors.ENDC)
  res_rev_sorted = test_runtime(sort_function, lambda n: np.arange(n, 0, -1), input_sizes)
  print(bcolors.OKBLUE + "\t - Input is a random array:" + bcolors.ENDC)
  res_random = test_runtime(sort_function, np.random.rand, input_sizes)
  plt.plot(input_sizes, res_sorted, input_sizes, res_rev_sorted, input_sizes, res_random)
  plt.title("Execution runtime of " + sort_function.__name__)
  plt.legend(["Sorted input", "Reverse sorted input", "Random input"])
  plt.xlabel("Input size")
  plt.ylabel("Execution time (ms)")
  plt.show()

def test_unique(fun):
    import random
    random.seed()
    unitTest = UnitTest(fun, post_process=sorted)

    unitTest.add_input([], [])
    unitTest.add_input([0], [0, 0, 0, 0])
    unitTest.add_input([-1, 3, 4, 11, 22], [4, 3, 22, 3, 11, 4, -1 , 3, -1])
    for i in range(5):
        a = random.choices(range(10), k=10)
        truth = list(set(a))
        unitTest.add_input(truth, a)
    return unitTest.run()

def test_inter(fun):
    import random
    random.seed()
    unitTest = UnitTest(fun, post_process=sorted)

    unitTest.add_input([], [], [])
    unitTest.add_input([], [0, 1], [])
    unitTest.add_input([], [], [0, 1])
    
    for i in range(5):
        a = random.choices(range(10), k=10)
        b = random.choices(range(10), k=10)
        truth = list(set(a).intersection(set(b)))
        unitTest.add_input(truth, a, b)
    return unitTest.run()

def test_is_balanced(fun):
    unitTest = UnitTest(fun)

    unitTest.add_input(True, "")
    unitTest.add_input(True, "()")
    unitTest.add_input(True, "[]")
    unitTest.add_input(False, "([)]")
    unitTest.add_input(True, "([])")
    unitTest.add_input(False, "([")
    unitTest.add_input(True, "[()[]]")
    unitTest.add_input(False, ")")
    return unitTest.run()

def test_reverse_polish_evaluation(fun):
    unitTest = UnitTest(fun)

    unitTest.add_input(20, "3 2 + 4 *")
    unitTest.add_input(11, "3 2 4 * +")
    unitTest.add_input(18, "3 2 4 + *")
    unitTest.add_input(20, "3 2 4 + * 2 +")
    unitTest.add_input(15, "3 2 4 + 2 * +")
    return unitTest.run()

def unit_test_dynamic_array(c):
  import math
  for strategy in ["linear", "geometric"]:
    da = c(strategy)
    unitTest = UnitTestImmediate("Strategy: " + strategy)
    unitTest.start()
    for i in range(1000):
      da.insert_back(i)
      unitTest.assert_true(lambda : da.size() == i + 1, "Size of the dynamic array " + str(da.size()) + " is incorrect (expected " + str(i + 1) + ")", "size")
      unitTest.assert_true(lambda : da.at(i) == i, "Content of the dynamic array " + str(da.at(i)) + " is incorrect (expected " + str(i) + ")", "at")
      if i == 0:
        unitTest.assert_true(lambda : da.capacity() == 1, "Initial capacity of the dynamic array is incorrect", "capacity")
      else:
        if strategy == "linear":
          excepted_capacity = 1 + 5 * math.ceil(i/5)  
        else:
          excepted_capacity = 2 ** (1 + math.floor(math.log2(i)))
        unitTest.assert_true(lambda : da.capacity() == excepted_capacity, "Capacity of the dynamic array " + str(da.capacity()) + " is incorrect (expected " + str(excepted_capacity) + ")", "capacity")  

    unitTest.end()

def test_dynamic_array_runtime(c, sizes, strategy, iter=5):
  """
  Function to test the runtime of insertions in the dynamic array
  """
  res = []
  for n in sizes:
    tmp = []
    for i in range(iter):
      da = c(strategy)
      t1 = time.time()
      for i in range(n):
        da.insert_back(n)
      t2 = time.time()
      tmp.append((t2 - t1) / n)
    tmp.sort()
    t = tmp[len(tmp) // 2] * 1000
    res.append(t)
    print("\t\t", n, "->", t)
  return res


def plot_dynamic_array_runtime(c):
  sizes = [i * 1000 for i in range(1, 71, 3)]

  print("Dynamic array ")
  print("\tLinear allocation")
  linear = test_dynamic_array_runtime(c, sizes, "linear")
  print("\tGeometric allocation")
  geometric = test_dynamic_array_runtime(c, sizes, "geometric")

  plt.plot(sizes, linear, sizes, geometric)
  plt.title("Dynamic array: insertion time per element")
  plt.legend(["Linear allocation", "Geometric allocation"])
  plt.xlabel("Number of elements")
  plt.ylabel("time (ms)")
  plt.show()

def test_dequeue(c):
  de = c()
  unitTest = UnitTestImmediate()
  unitTest.start()
  unitTest.assert_true(lambda : de.size() == 0, "Initial size should be 0", "size")
  de.insert_front(1)
  unitTest.assert_true(lambda : de.size() == 1, "Size should be 1", "size")
  unitTest.assert_true(lambda : de.peek_front() == 1, "Front element should be 1", "peek_front")
  unitTest.assert_true(lambda : de.peek_back() == 1, "Back element should be 1", "peek_back")
  de.insert_front(2)
  unitTest.assert_true(lambda : de.size() == 2, "Size should be 2", "size")
  unitTest.assert_true(lambda : de.peek_front() == 2, "Front element should be 2", "peek_front")
  unitTest.assert_true(lambda : de.peek_back() == 1, "Back element should be 1", "peek_back")
  de.remove_front()
  unitTest.assert_true(lambda : de.size() == 1, "Size should be 1", "size")
  unitTest.assert_true(lambda : de.peek_front() == 1, "Front element should be 1", "peek_front")
  unitTest.assert_true(lambda : de.peek_back() == 1, "Back element should be 1", "peek_back")
  de.remove_back()
  unitTest.assert_true(lambda : de.size() == 0, "Size should be 0", "size")
  for i in range(6):
      de.insert_front(i)
  for i in range(6, 8):
      de.insert_back(i)
  unitTest.assert_true(lambda : de.size() == 8, "Size should be 8", "size")
  unitTest.assert_true(lambda : de.peek_front() == 5, "Front element should be 5", "peek_front")
  unitTest.assert_true(lambda : de.peek_back() == 7, "Back element should be 7", "peek_back")
  unitTest.end()

def test_reverse_single_linked_list(fun, c):
  
    def linked_list_to_list(n):
        res = []
        while n != None:
            res.append(n.value)
            n = n.next
        return res
    
    unitTest = UnitTest(fun, post_process=linked_list_to_list, show_correct=False)
    
    n = None
    for i in range(8, -1, -1):
        n = c(i, n)

    expected = None
    for i in range(9):
        expected = c(i, expected)

    unitTest.add_input(expected, n)

    return unitTest.run()


def test_merge_sorted_arrays(fun):
    unitTest = UnitTest(fun)

    unitTest.add_input([], [], [])
    unitTest.add_input([1, 2], [1, 2], [])
    unitTest.add_input([1, 2], [], [1, 2])
    unitTest.add_input([1, 2, 3, 4], [1, 3], [2, 4])
    unitTest.add_input([1, 1, 2, 3], [1, 2, 3], [1])
    unitTest.add_input([1, 1, 2, 3], [1], [1, 2, 3])
    return unitTest.run()

def test_reorder_array(fun):

    import numpy as np
    np.random.seed(0)
    unitTest = UnitTestImmediate()
    unitTest.start()

    for i in range(200):
        array = np.random.randint(0, 100, size=(200,))
        pivot = array[-1]
    
        i = fun(array)
        
        unitTest.assert_true(lambda : i is not None and array[i] == pivot, "Reorder array does not work: the pivot index is incorrect", "reorder_array")
        unitTest.assert_true(lambda : np.all(array[:i] <= pivot), "Reorder array does not work, some elements before the pivot are greater than the pivot.", "reorder_array")
        unitTest.assert_true(lambda : np.all(array[i:] >= pivot), "Reorder array does not work, some elements after the pivot are smaller than the pivot.", "reorder_array")

    unitTest.end()


def unit_test_square_matrix_multiplication(fun):
    import numpy as np
    np.random.seed(0)
    unitTest = UnitTest(fun, test_fun=lambda x, y: np.allclose(x, y), show_correct=False)

    for i in range(3):
        a = np.random.randint(0, 10, size=(4, 4))
        b = np.random.randint(0, 10, size=(4, 4))
        c = np.dot(a, b)
        unitTest.add_input(c, a, b)

    size = 2**7
    a = np.random.rand(size, size)
    b = np.random.rand(size, size)
    c = np.dot(a, b)
    unitTest.add_input(c, a, b)

    return unitTest.run()

def test_change_making(fun):
    unitTest = UnitTest(fun)
    unitTest.add_input([5], [5, 2, 1], 5)
    unitTest.add_input([5, 1], [5, 2, 1], 6)
    unitTest.add_input([2, 2], [5, 2, 1], 4)
    unitTest.add_input([2, 1], [5, 2, 1], 3)
    unitTest.add_input([50, 20, 10, 5, 1], [200, 100, 50, 20, 10, 5, 2, 1], 86)
    unitTest.add_input([100, 50, 20, 5, 2], [200, 100, 50, 20, 10, 5, 2, 1], 177)

    return unitTest.run()

def test_activity_selection(fun):
    
    unit_test = UnitTest(fun)

    unit_test.add_input([(0, 1)], [(0, 1)])
    unit_test.add_input([(0, 1), (1, 2)], [(1, 2), (0, 1)])
    unit_test.add_input([(0, 2), (2, 4)], [(1, 3), (0, 2), (2, 4)])

    start=  [0, 6, 2, 1, 8, 3, 5, 3]
    finish= [3, 7, 6, 2, 9, 5, 9, 6]
    activities = list(zip(start, finish))
    import numpy as np
    np.random.seed(0)
    np.random.shuffle(activities)
    unit_test.add_input([(1, 2), (3, 5), (6, 7), (8, 9)], activities)

    return unit_test.run()

def test_egyptian_fraction(fun):
    unit_test = UnitTest(fun)

    unit_test.add_input([], 0, 1)
    unit_test.add_input([2], 1, 2)
    unit_test.add_input([2, 6], 2, 3)
    unit_test.add_input([2, 3, 12, 156], 12, 13)
   
    return unit_test.run()

def test_decode_one_symbol(fun, Node):
    unit_test = UnitTest(fun)

    n1 = Node(5, "a")
    n2 = Node(6, "b")
    n3 = Node(10, "c")
    n4 = Node(11, left_child=n1, right_child=n2)
    root = Node(21, left_child=n3, right_child=n4)

    unit_test.add_input(("c", 1), root, "010")
    unit_test.add_input(("b", 2), root, "11010")

    return unit_test.run()

def test_decode_all(fun, Node):
    unit_test = UnitTest(fun)

    n1 = Node(5, "a")
    n2 = Node(6, "b")
    n3 = Node(10, "c")
    n4 = Node(11, left_child=n1, right_child=n2)
    root = Node(21, left_child=n3, right_child=n4)

    unit_test.add_input("ca", root, "010")
    unit_test.add_input("bcac", root, "110100")

    return unit_test.run()

def test_frequency(fun):
    def test_fun(x, y):
        if x is None:
           return False
        return set(zip(*x)) == set(zip(*y))
    
    unit_test = UnitTest(fun, test_fun=test_fun)
    unit_test.add_input((["b","a"], [2,1]), "bab")
    unit_test.add_input((["a","b","c","e"], [5,3,2,8]), "abeeebaceaeeabeeca")

    return unit_test.run()

def test_huffman_tree(fun, Node):
    unit_test = UnitTest(fun)
    n1 = Node(5, "a")
    n2 = Node(3, "b")
    n3 = Node(2, "c")
    n4 = Node(8, "e")
    n5 = Node(5, '', n3, n2)
    n6 = Node(10, '', n1, n5)
    root = Node(18, '', n4, n6)

    unit_test.add_input(root, "abeeebaceaeeabeeca")
    return unit_test.run()

def test_codewords(fun, Node):
    unit_test = UnitTest(fun)
    n1 = Node(5, "a")
    n2 = Node(3, "b")
    n3 = Node(2, "c")
    n4 = Node(8, "e")
    n5 = Node(5, '', n3, n2)
    n6 = Node(10, '', n1, n5)
    root = Node(18, '', n4, n6)

    unit_test.add_input({'b': '111', 'c': '110', 'a': '10', 'e': '0'}, root)
    return unit_test.run()

def test_encode(fun, Node):
    unit_test = UnitTest(fun)
    n1 = Node(5, "a")
    n2 = Node(3, "b")
    n3 = Node(2, "c")
    n4 = Node(8, "e")
    n5 = Node(5, '', n3, n2)
    n6 = Node(10, '', n1, n5)
    root = Node(18, '', n4, n6)

    unit_test.add_input((root, "101110001111011001000101110011010"), "abeeebaceaeeabeeca")
    return unit_test.run()

### Change making problem

**Implement a greedy algorithm for the change making problem.**

The unit test must says that your function seems correct.

In [57]:
def change_making(coin_set, value):
    """
    Determines the minimum number of coins needed to make a given value using a greedy algorithm.

    Args:
      coin_set (list): A list of coin denominations available.
      value (int): The value to be made with the coins.

    Returns:
      list: A list of coins that sum up to the given value. The list contains the denominations of the coins used.
    """
    res = []
    coin_set.sort(reverse=True)
    for coin in coin_set:
        while value // coin > 0:
            res.append(coin)
            value -= coin
    return res

test_change_making(change_making)

--- Running test: 
âœ”  change_making([5, 2, 1], 5) -> [5]
âœ”  change_making([5, 2, 1], 6) -> [5, 1]
âœ”  change_making([5, 2, 1], 4) -> [2, 2]
âœ”  change_making([5, 2, 1], 3) -> [2, 1]
âœ”  change_making([200, 100, 50, 20, 10, 5, 2, 1], 86) -> [50, 20, 10, 5, 1]
âœ”  change_making([200, 100, 50, 20, 10, 5, 2, 1], 177) -> [100, 50, 20, 5, 2]
âœ” 6/6 tests passed.
--- Test finished.


True

### Activity selection problem

**Implement a greedy solution to the activity selection problem.**

The unit test must says that your function seems correct.

In [58]:
def activity_selection(activities):
		"""
		Input: list of activities as pairs (start, finish)
		Output: maximal list of non overlapping activities

		Example:
		>>activity_selection([(0, 3), (6, 7), (2, 6), (1, 2), (8, 9), (3, 5), (5, 9), (3, 6)])
		[(1, 2), (3, 5), (6, 7), (8, 9)]
		"""
		if len(activities) == 0:
			return []
		sorted_activities = sorted(activities, key=lambda x: x[1])
		selection = [sorted_activities[0]]
		for i in range(1, len(sorted_activities)):
			if sorted_activities[i][0] >= selection[-1][1]:
				selection.append(sorted_activities[i])
		return selection

test_activity_selection(activity_selection)

--- Running test: 
âœ”  activity_selection([(0, 1)]) -> [(0, 1)]
âœ”  activity_selection([(1, 2), (0, 1)]) -> [(0, 1), (1, 2)]
âœ”  activity_selection([(1, 3), (0, 2), (2, 4)]) -> [(0, 2), (2, 4)]
âœ”  activity_selection([(5, 9), (2, 6), (6, 7), (3, 6), (1, 2), (0, 3), (3, 5), (8, 9)]) -> [(1, 2), (3, 5), (6, 7), (8, 9)]
âœ” 4/4 tests passed.
--- Test finished.


True

### Egyptian Fraction

**Implement the Egyptian Fraction decomposition algorithm.**

**You must keep an explicit representation of the fractions with their numerator and denominator in the implementation**

The unit test must says that your function seems correct.

In [59]:
from math import ceil

def egyptian_fraction(a, b):
    """
    Compute the Egyptian fraction decomposition of the fraction a / b with 0 < a < b
    Return the list of denominators of the decomposition in increasing order

    Example:
    >>egyptian_fraction(12, 13)
    [2, 3, 12, 156]
    """
    denominators = []
    while a != 0:
        next_denominator = ceil(b/a)
        denominators.append(next_denominator)
        a = a * next_denominator - b
        b = b * next_denominator
    return denominators

test_egyptian_fraction(egyptian_fraction)

--- Running test: 
âœ”  egyptian_fraction(0, 1) -> []
âœ”  egyptian_fraction(1, 2) -> [2]
âœ”  egyptian_fraction(2, 3) -> [2, 6]
âœ”  egyptian_fraction(12, 13) -> [2, 3, 12, 156]
âœ” 4/4 tests passed.
--- Test finished.


True

### Huffman coding

The following class represents a Node of a binary tree for Huffman coding.

In [60]:
class Node:
    """
    Node of a binary tree. A node has 4 attributes:
    - weight (mandatory)
    - symbol (optional)
    - left_child (optional)
    - right_child (optional)
    """

    def __init__(self, weight, symbol='', left_child=None, right_child=None):
        self.weight = weight
        self.symbol = symbol
        self.left_child = left_child
        self.right_child = right_child

    def __repr__(self):
        """
        String representation of the node: (weight, symbol, left_child, right_child)
        """
        def str_rec(node):
            if node is None:
                return ""
            else:
                return str(node)
        return "(" + str(self.weight) + ", " + str(self.symbol) + ", " + str_rec(self.left_child) + ", " + str_rec(self.right_child) +")"
    
    def __eq__(self, other):
        """
        Comparison function == between two nodes.
        """
        if not(isinstance (other, Node)):
            return False
        return self.weight == other.weight and self.symbol == other.symbol and self.left_child == other.left_child and self.right_child == other.right_child

    def __lt__(self, other):
        """
        Comparison function < between two nodes.
        """

        return self.weight < other.weight
    
def print_demo_tree():
    """
    Construct and print a demo tree
    """
    n1 = Node(5, "a")
    n2 = Node(6, "b")
    n3 = Node(10, "c")
    n4 = Node(11, left_child=n1, right_child=n2)
    root = Node(21, left_child=n3, right_child=n4)
    print("Displays a small tree:")
    print(root)

print_demo_tree()


Displays a small tree:
(21, , (10, c, , ), (11, , (5, a, , ), (6, b, , )))


**Implement the function decode_one_symbol**

The unit test must says that your function seems correct.

In [61]:
def decode_one_symbol(root: Node, binary_string):
    """
    Decode the first symbol in the binary string with the tree 
    given by the root node.

    Returns the decoded symbol and the length of the corresponding code

    Example:

    >>n1 = Node(5, "a")
    >>n2 = Node(6, "b")
    >>n3 = Node(10, "c")
    >>n4 = Node(11, left_child=n1, right_child=n2)
    >>root = Node(21, left_child=n3, right_child=n4)
    >>decode_one_symbol(root, "010")
    ("c", 1)
    >>decode_one_symbol(root, "11010")
    ("b", 2)
    """
    depth = 0
    curr_node = root
    for b in binary_string:
        if curr_node.left_child is None and curr_node.right_child is None:
            return (curr_node.symbol, depth)
        if b == '0':
            curr_node = curr_node.left_child
        else:
            curr_node = curr_node.right_child
        depth += 1        
    return ()

test_decode_one_symbol(decode_one_symbol, Node);

--- Running test: 
âœ”  decode_one_symbol((21, , (10, c, , ), (11, , (5, a, , ), (6, b, , ))), 010) -> ('c', 1)
âœ”  decode_one_symbol((21, , (10, c, , ), (11, , (5, a, , ), (6, b, , ))), 11010) -> ('b', 2)
âœ” 2/2 tests passed.
--- Test finished.


**Implement the function decode_all**

The unit test must says that your function seems correct.

In [62]:
def decode_all(root: Node, binary_string: str):
    """
    Decode binary_string with the tree 
    given by the root node.

    Returns the decoded string

    Example:

    >>n1 = Node(5, "a")
    >>n2 = Node(6, "b")
    >>n3 = Node(10, "c")
    >>n4 = Node(11, left_child=n1, right_child=n2)
    >>root = Node(21, left_child=n3, right_child=n4)
    >>decode_all(root, "010")
    "ca"
    >>decode_all(root, "110100")
    "bcac"
    """
    decoded = []
    curr_node = root
    for b in binary_string:
        if curr_node.left_child is None and curr_node.right_child is None:
            decoded.append(curr_node.symbol)
            curr_node = root
        if b == '0':
            curr_node = curr_node.left_child
        else:
            curr_node = curr_node.right_child
    if curr_node.left_child is None and curr_node.right_child is None:
      decoded.append(curr_node.symbol)
    return "".join(decoded)

test_decode_all(decode_all, Node);

--- Running test: 
âœ”  decode_all((21, , (10, c, , ), (11, , (5, a, , ), (6, b, , ))), 010) -> ca
âœ”  decode_all((21, , (10, c, , ), (11, , (5, a, , ), (6, b, , ))), 110100) -> bcac
âœ” 2/2 tests passed.
--- Test finished.


**Implement the function frequency**

The unit test must says that your function seems correct.

In [63]:
def frequency(string):
    """
    Returns the list of symbols present in the given string and their frequency.

    Example:
    >>frequency("abeeebaceaeeabeeca")
    (["a","b","c","e"], [5,3,2,8])
    """
    letters_frequency = {}
    for letter in string:
        letters_frequency[letter] = letters_frequency.get(letter, 0) + 1
    return (list(letters_frequency.keys()), list(letters_frequency.values()))

test_frequency(frequency)

--- Running test: 
âœ”  frequency(bab) -> (['b', 'a'], [2, 1])
âœ”  frequency(abeeebaceaeeabeeca) -> (['a', 'b', 'c', 'e'], [5, 3, 2, 8])
âœ” 2/2 tests passed.
--- Test finished.


True

**Implement the function huffman_tree**

The unit test must says that your function seems correct.

In [64]:
import heapq
def huffman_tree(string):
    """
    Create the Huffman tree for the given string
    Returns the root of the tree.

    Child nodes are sorted by increasing weight: left_child <= right_child

    The Python module heapq is used to manipulate binary heaps:
    https://docs.python.org/3/library/heapq.html

    Example:
    >>huffman_tree("abeeebaceaeeabeeca")
    (18, , (8, e, , ), (10, , (5, a, , ), (5, , (2, c, , ), (3, b, , ))))

    """
    # Calculate the frequency of each character in the string
    freq = frequency(string)
    print(freq)
    freq = {freq[0][i]: freq[1][i] for i in range(len(freq[0]))}
    
    # Create a priority queue (min-heap) from the frequency dictionary
    heap = [Node(f, c) for c, f in freq.items()]
    heapq.heapify(heap)   

    # Build the Huffman tree
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(left.weight + right.weight,'' ,left_child=left, right_child=right)
        heapq.heappush(heap, merged)
    return heap[0]
  
test_huffman_tree(huffman_tree, Node);

--- Running test: 
(['a', 'b', 'e', 'c'], [5, 3, 8, 2])
âœ”  huffman_tree(abeeebaceaeeabeeca) -> (18, , (8, e, , ), (10, , (5, a, , ), (5, , (2, c, , ), (3, b, , ))))
âœ” 1/1 tests passed.
--- Test finished.


**Implement the function codewords**

The unit test must says that your function seems correct.

In [65]:
def codewords(root):
    """
    Creates a dictionary which associates any symbol of the given tree to its codeword.

    Example:
    >>n1 = Node(5, "a")
    >>n2 = Node(3, "b")
    >>n3 = Node(2, "c")
    >>n4 = Node(8, "e")
    >>n5 = Node(5, '', n3, n2)
    >>n6 = Node(10, '', n1, n5)
    >>root = Node(18, '', n4, n6)
    >>codewords(root)
    {'b': '111', 'c': '110', 'a': '10', 'e': '0'}
    """
    def traverse(node, path, code_dict):
        if node is not None:
            if node.symbol:  # Leaf node
                code_dict[node.symbol] = path
            traverse(node.left_child, path + '0', code_dict)
            traverse(node.right_child, path + '1', code_dict)
        return

    code_dict = {}
    traverse(root, '', code_dict)
    return code_dict


test_codewords(codewords, Node)

--- Running test: 
âœ”  codewords((18, , (8, e, , ), (10, , (5, a, , ), (5, , (2, c, , ), (3, b, , ))))) -> {'b': '111', 'c': '110', 'a': '10', 'e': '0'}
âœ” 1/1 tests passed.
--- Test finished.


True

**Implement the function encode**

The unit test must says that your function seems correct.

In [66]:
def encode(string):
    """
    Encode a string with the Huffman coding.
    Return the root of the Huffman tree and the encoded string

    Example:
    >>encode("abeeebaceaeeabeeca")
    ((18, , (8, e, , ), (10, , (5, a, , ), (5, , (2, c, , ), (3, b, , )))),"101110001111011001000101110011010")
    """
    root = huffman_tree(string)
    code_dict = codewords(root)
    encoded_string = ''.join(code_dict[char] for char in string)
    return root, encoded_string

test_encode(encode, Node)

--- Running test: 
(['a', 'b', 'e', 'c'], [5, 3, 8, 2])
âœ”  encode(abeeebaceaeeabeeca) -> ((18, , (8, e, , ), (10, , (5, a, , ), (5, , (2, c, , ), (3, b, , )))), '101110001111011001000101110011010')
âœ” 1/1 tests passed.
--- Test finished.


True

**Final sanity check**: decode_all(encode) must be the identity function

We also compute the compression ratio assuming that a character is encoded on 8 bits.

In [67]:
lorem_ipsum = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. " + \
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. " + \
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. " + \
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

for s in ["abeeebaceaeeabeeca", "bab", "abbbccddde", lorem_ipsum]:
    root, encoded = encode(s)
    decoded = decode_all(root, encoded)
    compression_ratio = (len(encoded) / 8) / len(s)
    assert s == decoded, "Error: decoded string is different from original string"
    print("decode_all(encode('" + s + "')) OK")
    print("\tcompression ratio ->", compression_ratio)
    

(['a', 'b', 'e', 'c'], [5, 3, 8, 2])
decode_all(encode('abeeebaceaeeabeeca')) OK
	compression ratio -> 0.22916666666666666
(['b', 'a'], [2, 1])
decode_all(encode('bab')) OK
	compression ratio -> 0.125
(['a', 'b', 'c', 'd', 'e'], [1, 3, 2, 3, 1])
decode_all(encode('abbbccddde')) OK
	compression ratio -> 0.275
(['L', 'o', 'r', 'e', 'm', ' ', 'i', 'p', 's', 'u', 'd', 'l', 't', 'a', ',', 'c', 'n', 'g', 'b', 'q', '.', 'U', 'v', 'x', 'D', 'h', 'f', 'E'], [1, 29, 22, 37, 17, 68, 42, 11, 18, 28, 18, 21, 32, 29, 4, 16, 24, 3, 3, 5, 4, 1, 3, 3, 1, 1, 3, 1])
decode_all(encode('Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit an